# 1. 분석기획


해당 프로젝트는 1912년 타이타닉호에 탑승했던 탑승객 중 일부 데이터 (생존여부 포함) 를 학습하여, 생존에 기여한 데이터 특성을 발견하고, 학습에 사용되지 않은 테스트 데이터에  대하여 생존율 예측을 90% 이상의 정확도를 가지는 머신러닝 모델 개발을 목표로 한다.  
별도의 도메인 지식은 필요하지 않으며 프로젝트상 리스크는 특별히 존재하지 않는다.

# 2. 데이터 준비

해당 프로젝트에서는 별도의 데이터 수집단계를 거치지 않으며, 분석에 필요한 데이터가 주어진 것을 가정한다.  
(실제 인공지능 시스템 구현 프로젝트에서는 분석기획에 따라 장기간 데이터의 축적/레이블링이 선행되어야 할 수 있다.)


### 라이브러리 Import

In [2]:
from google.colab import drive
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

### 데이터 로드

In [3]:
# 구글 드라이브 mount
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
!ls /content/gdrive/MyDrive/SeSAC/Lecture_1_AI/Data

titanic


In [13]:
# 데이터 파일 로드
colab_path= "/content/gdrive/MyDrive/SeSAC/Lecture_1_AI/Data/"
df_train = pd.read_csv(colab_path + 'titanic/titanic_train.csv')
df_test = pd.read_csv(colab_path + 'titanic/titanic_test.csv')
print("학습데이터: {}건, 테스트용데이터: {}건".format(len(df_train), len(df_test)))

학습데이터: 891건, 테스트용데이터: 418건


# 3. 데이터 분석

## 3.1. 분석용 데이터 준비: 전처리




### 데이터 구조 확인

In [ ]:
# 데이터 샘플 확인


In [ ]:
# 데이터 dimension 확인


In [ ]:
# 데이터의 개략적인 정보 확인


### Label 변수 확인

In [ ]:
# 'Survived' 필드 구성 값 확인


In [ ]:
# value_counts()는 Series의 unique value를 count해주는 함수


In [ ]:
# value_counts() 결과값을 percent 정보로 변환


### 결측치 처리

In [ ]:
# 결측치 확인


In [ ]:
# 결측치 처리 - Embarked


In [ ]:
# 결측치 처리(대체) - Embarked


In [ ]:
# 결측치 처리 - Age
# 'Age'와의 상관관계가 높은 Feature 확인


In [ ]:
# 결측치 처리 - Age
# 추후 Age가 결측치인 데이터를 필터링하여 추가분석 할 수 있도록 Column 생성


In [ ]:
# 결측치 처리 - Age
# 평균값으로 대체 (mode, median 값 등을 사용할 수도 있음)


In [ ]:
# 결측치 처리(대체) - Age


In [ ]:
df_train

In [ ]:
# 결측치 처리 - Cabin


In [ ]:
# 결측치 처리(해당 Feature 제외) - Cabin


In [ ]:
# 결측치 처리 결과 확인
df_train.isnull().sum()

### 이상값 처리

In [ ]:
# Pclass(등급) unique값


In [ ]:
# Sex(성별) unique값


In [ ]:
# SibSp(함께 탑승한 형제자매, 아내, 남편의 수) unique값


In [ ]:
# Parch(함께 탑승한 부모, 자식의 수) unique값


In [ ]:
# Age, Fare 값 BoxPlot으로 확인


In [ ]:
# Fare 데이터 자세히 들여다보기 - Pclass별로


In [ ]:
# Fare 데이터 자세히 들여다보기 - 실제 샘플 확인


In [ ]:
df_train

### 데이터 정규화

In [14]:
# Age, Fare에 대해 z-score 정규화
columns_to_scale = ['Age', 'Fare']
scaler_train = preprocessing.StandardScaler()
# 나중에 테스트 데이터에 적용하기 위하여 fit 단계와 transform 단계를 분리한다.
scaler_train = scaler_train.fit(df_train[columns_to_scale])
df_train[columns_to_scale] = scaler_train.transform(df_train[columns_to_scale])
df_train

### 데이터 인코딩

In [ ]:
# Sex(성별), Embarked(배에 탑승한 위치) 데이터 인코딩
df_train = pd.get_dummies(df_train, columns = ['Sex', 'Embarked'])
df_train.iloc[:, -5:]

In [ ]:
# Ticket 데이터 살펴보기
df_train['Ticket'].sort_values().values[-10:]

In [ ]:
# Name 데이터 살펴보기
df_train['Name'].sort_values().values[:10]

In [15]:
# Name에 포함된 호칭 추출하여 확인
# 정규표현식은 학습범위를 벗어나므로 결과만을 참고한다.
df_train['Name'].str.extract("([A-Za-z]+)\.").value_counts(dropna=False)


,count
0,
Mr,517
Miss,182
Mrs,125
Master,40
Dr,7
Rev,6
Mlle,2
Major,2
Col,2


In [ ]:
# 이름에 'Master', 'Dr', 'Rev(목사)', 'Major', 'Col(대령)', 'Capt', 'Jonkheer(귀족)', 'Don', 'Countess(백작)', 'Sir' 포함여부를 Flag화 한다.
df_train['Name_has_title'] = df_train['Name'].str.contains('Master|Dr|Rev|Major|Col|Capt|Jonkheer|Don|Countess|Sir').astype(int)

In [ ]:
# Name, Ticket 필드 분석에서 제외
df_train = df_train.drop(columns=['Ticket', 'Name'])

In [ ]:
df_train

## 3.2. 텍스트 분석

해당 프로젝트에서는  (3-1. 단계에서 Name 필드로부터 name_has_title Feature를 추출한 정도 외에는) 텍스트 마이닝이 필요할 정도의 비정형 데이터를 포함하지 않아 텍스트 분석 단계를 거치지 않는다.

## 3.3. 탐색적 분석

### 산포도 분석


In [ ]:
# Pclass에 따른 Survived(사망자/생존자) 분포
hist1 = df_train[df_train['Survived']==0]['Pclass']
hist2 = df_train[df_train['Survived']==1]['Pclass']
plt.hist([hist1, hist2], label=['Dead', 'Alive'], alpha=0.5)
plt.legend(loc=0)

In [ ]:
# Age에 따른 Survived(사망자/생존자) 분포
hist1 = df_train[df_train['Survived']==0]['Age']
hist2 = df_train[df_train['Survived']==1]['Age']
plt.hist(hist1, label='Dead', bins=25, alpha=0.5)
plt.hist(hist2, label='Alive', bins=25, alpha=0.5)
plt.legend(loc=0)

### 상관관계 분석

In [ ]:
# 상관관계 계산
cols_to_corr = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_has_title']
df_train[cols_to_corr].corr().round(2)

In [ ]:
# 상관관계 시각화
cols_to_corr = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_has_title']
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(df_train[cols_to_corr].corr().round(2), cmap='RdBu_r')
fig.colorbar(c, ax=ax)
plt.xticks(np.arange(0.5, len(cols_to_corr), 1), cols_to_corr)
plt.yticks(np.arange(0.5, len(cols_to_corr), 1), cols_to_corr)

### 전처리 일괄 실행 함수 만들기

## 3.4. 모델링

In [ ]:
# 전처리 함수 refactoring
def part5_preprocessing(df_orig, mode='train', replace_embarked=None, replace_age=None, scaler=None):
    df = df_orig.copy()

    # 결측치 처리 (훈련 데이터 상에서 파악한 최빈도값/평균값 공통 사용)
    if mode == 'train':
        replace_embarked = df_train['Embarked'].value_counts(dropna=False).idxmax()
        replace_age = df_train['Age'].mean().round(2)
    df['Embarked'] = df['Embarked'].fillna(replace_embarked)
    df['Age_isna'] = df['Age'].isna().astype(int)
    df['Age'] = df['Age'].fillna(replace_age)

    # 이상값 처리 - 없음

    # 데이터 정규화 (훈련 데이터 상에서 학습된 scaler 공통 사용)
    if mode == 'train':
        scaler = preprocessing.StandardScaler()
        scaler = scaler.fit(df[['Age', 'Fare']])
    df[['Age', 'Fare']] = scaler.transform(df[['Age', 'Fare']])

    # 데이터 인코딩
    df = pd.get_dummies(df, columns = ['Sex', 'Embarked'])
    df['Name_has_title'] = df['Name'].str.contains('Master|Dr|Rev|Major|Col|Capt|Jonkheer|Don|Countess|Sir').astype(int)

    # 전처리에 사용된 원본 필드, 분석에 불필요한 필드를 제외시킨다.
    df = df.drop(columns=['Name', 'Ticket', 'Cabin'])

    return df, replace_embarked, replace_age, scaler

In [ ]:
# 학습 데이터 전처리 (일괄 적용)
df_train = pd.read_csv(colab_path + 'titanic/titanic_train.csv')
df_train, replace_embarked, replace_age, scaler = part5_preprocessing(df_train,
'train')


In [ ]:
# Feature(X), Label(y) 분리
target_col = 'Survived'
train_y = df_train[target_col]
train_X = df_train.drop(columns=[target_col])
train_X = train_X.set_index('PassengerId')

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련용/평가용 데이터 분할
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y,
test_size=0.3, random_state=42 )
train_X

In [ ]:
# 모델 1 학습
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(C=20, max_iter=1000, random_state=42)
model1.fit(train_X, train_y)

In [ ]:
# 모델 1 평가
print(model1.score(train_X, train_y)) # accuracy
print(model1.score(val_X, val_y))

In [ ]:
# 모델 2 학습
from sklearn.svm import SVC
model2 = SVC(kernel = 'linear', probability=True, random_state=42)
model2.fit(train_X, train_y)

In [ ]:
# 모델 2 평가
print(model2.score(train_X, train_y)) # accuracy
print(model2.score(val_X, val_y))

In [ ]:
# 모델 3 학습
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(max_depth = 3, random_state=42)
model3.fit(train_X, train_y)

In [ ]:
# 모델 3 평가
print(model3.score(train_X, train_y)) # accuracy
print(model3.score(val_X, val_y))

## 3.5. 모델 평가 및 검증

In [ ]:
# ROC Curve 플롯팅
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(val_y, model1.predict_proba(val_X)[:,1])
plt.plot([0,1], [0,1], "k--", "r+") #0.5 기준선
plt.plot(fpr, tpr, label='Logistic Regression')
fpr2, tpr2, thresholds2 = roc_curve(val_y, model2.predict_proba(val_X)[:,1])
plt.plot(fpr2, tpr2, label='SVM')
fpr3, tpr3, thresholds3 = roc_curve(val_y, model3.predict_proba(val_X)[:,1])
plt.plot(fpr3, tpr3, label='RandomForest')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend(loc=0)

In [ ]:
# AUC Score
from sklearn.metrics import roc_auc_score
print('AUC value (Logistic Regression): {}'.format(roc_auc_score(val_y,
model1.predict(val_X))))
print('AUC value (SVM): {}'.format(roc_auc_score(val_y, model2.predict(val_X))))
print('AUC value (RandomForest): {}'.format(roc_auc_score(val_y,
model3.predict(val_X))))


## 3.6. 모델 적용 및 운영방안 수립 (테스트 데이터셋 예측)

In [ ]:
# 모델 저장
import pickle
# AUC Score 기준 가장 성능이 좋았던 Logistic Regression 모델을 저장한다.

with open(colab_path + 'saved_model.pickle','wb') as fw:
    pickle.dump(model1, fw)

In [ ]:
# 모델 로드
with open(colab_path + 'saved_model.pickle','rb') as f:
    model = pickle.load(f)

In [ ]:
# 테스트 데이터 전처리
df_test = pd.read_csv(colab_path + '/titanic/titanic_test.csv')
df_test, replace_embarked, replace_age, scaler = part5_preprocessing(df_test,
'test', replace_embarked, replace_age, scaler)  # 위에서 만들어놓은 refactorying method 이용
df_test

In [ ]:
# 테스트 데이터 중 1건에 'Fare'에 대한 결측치가 존재하므로 결측치를 train 데이터의 평균값으로 대체한다.
replace_fare = df_train['Fare'].mean().round(2)
df_test['Fare'] = df_test['Fare'].fillna(replace_fare)
test_X = df_test.set_index('PassengerId')

In [ ]:
# PassengerId 인덱스화

In [ ]:
# 최종 생존율 예측
pred_y = model.predict(test_X)
df_result = pd.DataFrame({'PassengerId':df_test['PassengerId'], 'Survived':
pred_y})
df_result

# 4. 시스템 구현 (Developing)

해당 프로젝트에서는 시스템 구현 단계는 수행하지 않는다. (전문교육 과정 마지막 프로젝트에서 시스템 구현에 대한 내용을 다룬다.)

# 5. 평가 및 전개 (Deploying)

자신만의 모델 개선

DACON에 다른 참가자 혹은 주최 측에서 제공한 baseline 코드들을 분석하고 차이점을 비교 및 내 모델 학습 알고리즘 개선
https://dacon.io/competitions/open/235539/codeshare